In [1]:
### Import packages 
import networkx as nx # for handling graphs/networks 
import numpy as np # for basic scientific computing 
import pandas as pd # for basic scientific computing 
import matplotlib.pyplot as plt # for plotting
import matplotlib.gridspec as gridspec
from scipy.integrate import ode
from sklearn.cluster import KMeans

import seaborn as sns

from scipy.special import binom
from scipy.linalg import subspace_angles

from scipy.integrate import solve_ivp
import Koopman_source as kp
from math import comb
#from copy import copy
import csv  
from scipy.optimize import minimize
from scipy.special import expit
from numpy.linalg import matrix_power


In [2]:
np.random.seed(123)

## Compute a Koopman matrix from simulated data.

#x0 = [2,1]

model = "Duffing"
sequential = False

if model == "d2_ans":
    d = 2
elif model == "d2":
    d = 2 # the number of variables (=the dimension) of the original dynamical system.
elif model == "d3":
    d = 3
elif model == "Lorenz":
    d = 3
elif model == "Lotka_Voltera":
    d = 2
elif model == "Duffing":
    d = 2

#M = 100 # the number of observed data. 

orth_basis = True

M_max = 2000 # the maximum number of observed data. 
n_repeat = 2 # the number of repetition of approximaing the Koopman matrix. 
max_order = 7
N = comb(d+max_order,d) # the total number of observable functions, psi_m's. 
# N1 = 4 
# N2 = N - N1

In [3]:

alpha0 = -1
beta0 = 1
delta0 = 0.3 # Damping parameter
# delta0 = 0.0 # Damping parameter
dt = 0.05

# Set parameters
M1, M2 = 10**2, 5  # number of data points
M_max = M1*M2
N = 100

x_mat_total = np.zeros([d,M_max])
y_mat_total = np.zeros([d,M_max])

for m1 in range(M1):
    x_mat_total[:, m1*M2] = np.random.uniform(-2, 2, size=(d, ))
    for m2 in range(M2-1):
        x_mat_total[:, m1*M2+m2+1] = kp.Duffing(x_mat_total[:, m1*M2+m2], alpha0=alpha0, beta0 = beta0, delta0 = delta0, dt = dt)

K_approx = np.zeros([N,N,M_max])
G_approx = np.zeros([N,N,M_max])
G_pinv = np.zeros([N,N,M_max])
A_approx = np.zeros([N,N,M_max])

for i in range(M_max):
    y_mat_total[:,i] = kp.Duffing(x_mat_total[:,i], alpha0=alpha0, beta0 = beta0, delta0 = delta0, dt = dt)


# Scaling for radial function
gamma_inv_scale = np.mean(np.linalg.norm(x_mat_total - np.mean(x_mat_total, axis=1)[:, np.newaxis], axis=0))

# Find centers using KMeans
kmeans = KMeans(n_clusters=N, random_state=0).fit(np.vstack((x_mat_total.T, y_mat_total.T)))
Cj = kmeans.cluster_centers_


Psi_X = np.zeros([M_max,N])
Psi_Y = np.zeros([M_max,N])

for i in range(M_max):
    for j in range(N):
        Psi_X[i,j] = kp.psi_radial(x_mat_total[:,i], Cj[j,:], 1/gamma_inv_scale)
        Psi_Y[i,j] = kp.psi_radial(y_mat_total[:,i], Cj[j,:], 1/gamma_inv_scale)

G_approx[:,:,0] = np.matmul(Psi_X[np.ix_(range(1),range(N))].transpose(), Psi_X[np.ix_(range(1),range(N))])
G_pinv[:,:,0] = np.linalg.pinv(G_approx[:,:,0])
A_approx[:,:,0] = np.matmul(Psi_X[np.ix_(range(1),range(N))].transpose(), Psi_Y[np.ix_(range(1),range(N))])
K_approx[:,:,0] = np.matmul(G_pinv[:,:,0], A_approx[:,:,0])

for M in range(2,M_max+1):  
    G_approx[:,:,M-1] = G_approx[:,:,M-2] + np.matmul(Psi_X[np.ix_(range(M-1,M),range(N))].transpose(), Psi_X[np.ix_(range(M-1,M),range(N))])
    G_pinv[:,:,M-1] = np.linalg.pinv(G_approx[:,:,M-1])
    A_approx[:,:,M-1] = A_approx[:,:,M-2] + np.matmul(Psi_X[np.ix_(range(M-1,M),range(N))].transpose(), Psi_Y[np.ix_(range(M-1,M),range(N))])
    K_approx[:,:,M-1] = np.matmul(G_pinv[:,:,M-1], A_approx[:,:,M-1])


/opt/anaconda3/envs/py_base/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
plt.scatter(x_mat_total[0,:],x_mat_total[1,:])
# plt.scatter(y_mat_total[0,:],y_mat_total[1,:])
plt.scatter(Cj[:,0],Cj[:,1])
# x_mat_total.shape



In [ ]:
mixed_idx = np.random.permutation(N)

K_approx_one = K_approx[:,:,M_max-1]
K_approx_one = 1/dt * (np.squeeze(K_approx_one) - np.identity(N))

eig_v = kp.pagerank(abs(K_approx_one), 50000, 0.7)
eig_v2 = kp.subsampled_pagerank(abs(K_approx_one), 50000, 0.7, 0.3)
ranked_idx = np.argsort(eig_v)[::-1]
ranked_idx2 = np.argsort(eig_v2)[::-1]

K_approx_one_mx = K_approx_one[np.ix_(mixed_idx,mixed_idx)]
# K_approx_one_mx = 1/dt * (np.squeeze(K_approx_one_mx) - np.identity(N))

K_approx_one_pr = K_approx_one[np.ix_(ranked_idx,ranked_idx)]
# K_approx_one_pr = 1/dt * (np.squeeze(K_approx_one_pr) - np.identity(N))

K_approx_one_subpr = K_approx_one[np.ix_(ranked_idx2,ranked_idx2)]
# K_approx_one_subpr = 1/dt * (np.squeeze(K_approx_one_subpr) - np.identity(N))

# Create a heatmap
#fig2, ax2 = plt.subplots(len(i_list), len(j_list), figsize = (8,8))
fig1, ax1 = plt.subplots(1,3,figsize = (15,4))
#plt.figure(figsize=(8, 6))
color_scale = 5
sns.heatmap(expit(K_approx_one*color_scale), ax=ax1[0], annot=True, cmap='coolwarm', cbar=True)
sns.heatmap(expit(K_approx_one_mx*color_scale), ax=ax1[1], annot=True, cmap='coolwarm', cbar=True)
sns.heatmap(expit(K_approx_one_pr*color_scale), ax=ax1[2], annot=True, cmap='coolwarm', cbar=True)# sns.heatmap(expit(data_subpr*10), ax=ax1[2], annot=True, cmap='coolwarm', cbar=True)
# sns.heatmap(expit(K_approx_one_pr*100), annot=True, cmap='coolwarm', cbar=True)
# sns.heatmap(1/(1+np.exp(-K_approx_one)), annot=True, cmap='coolwarm', cbar=True)

# Add title and labels
plt.title("Heatmap Example")
plt.xlabel("Columns")
plt.ylabel("Rows")


In [ ]:
# Computing the subspace angles with increasing number of (ordered) basis functions.

subsp_ang_mx = np.zeros(N)
subsp_ang_inc = np.zeros(N)
subsp_ang_pr = np.zeros(N)
subsp_ang_subpr = np.zeros(N)

M_ang = 1000
# dataX_for_angle = np.random.default_rng().exponential(scale=1, size=[d,M_ang])
# dataY_for_angle = np.random.default_rng().exponential(scale=1, size=[d,M_ang])
dataX_for_angle = np.random.default_rng(12).uniform(-2, 2, size=(d, M_ang))
dataY_for_angle = np.random.default_rng(13).uniform(-2, 2, size=(d, M_ang))
# 

Psi_X_ang = np.zeros([M_ang,N])
Psi_Y_ang = np.zeros([M_ang,N])

for i in range(M_ang):
    dataY_for_angle[:,i] = kp.Duffing(dataX_for_angle[:,i], alpha0=alpha0, beta0 = beta0, delta0 = delta0, dt = dt)

for i in range(M_ang):
    for j in range(N):
        Psi_X_ang[i,j] = kp.psi_radial(dataX_for_angle[:,i], Cj[j,:], 1/gamma_inv_scale)
        Psi_Y_ang[i,j] = kp.psi_radial(dataY_for_angle[:,i], Cj[j,:], 1/gamma_inv_scale)

for N_vary in range(2,N):
    # x_mat_total = np.random.default_rng().exponential(scale=1, size=[d,M_max])
    # Psi_Y_ang = np.matmul(Psi_X_ang[np.ix_(range(M_ang),range(N_vary))], data[np.ix_(range(N_vary),range(N_vary))])
    subsp_ang_inc[N_vary] = subspace_angles(Psi_X_ang[np.ix_(range(M_ang),range(N_vary))], Psi_Y_ang[np.ix_(range(M_ang),range(N_vary))])[0]
    subsp_ang_mx[N_vary] = subspace_angles(Psi_X_ang[np.ix_(range(M_ang),mixed_idx[range(N_vary)])], Psi_Y_ang[np.ix_(range(M_ang),mixed_idx[range(N_vary)])])[0]
    subsp_ang_pr[N_vary] = subspace_angles(Psi_X_ang[np.ix_(range(M_ang),ranked_idx[range(N_vary)])], Psi_Y_ang[np.ix_(range(M_ang),ranked_idx[range(N_vary)])])[0]
    subsp_ang_subpr[N_vary] = subspace_angles(Psi_X_ang[np.ix_(range(M_ang),ranked_idx2[range(N_vary)])], Psi_Y_ang[np.ix_(range(M_ang),ranked_idx2[range(N_vary)])])[0]



In [ ]:
# plt.plot(subsp_ang_inc * 180/np.pi)
plt.plot(subsp_ang_mx * 180/np.pi)
plt.ylim([0, 100])
plt.yticks([0, 30, 60, 90])
plt.xlabel("# of basis")
plt.ylabel("Subspace angle")
plt.plot(subsp_ang_pr * 180/np.pi)
# plt.plot(subsp_ang_subpr * 180/np.pi)
# plt.xlabel("Basis functions")
# plt.ylabel("PageRank")
plt.legend(["Mixed", "PageRank", "Sub-PageRank"])
plt.grid(True, linestyle='--', alpha=0.7)


In [ ]:
plt.scatter(x_mat_total[0,:],x_mat_total[1,:])
# plt.scatter(y_mat_total[0,:],y_mat_total[1,:])
plt.scatter(Cj[np.ix_(ranked_idx[range(10)], range(1))],Cj[np.ix_(ranked_idx[range(10)], range(1,2))])
# x_mat_total.shape



In [ ]:
# Computing the errro function by evoling a function with the true dynamics and approximated dynamics.
# Error: ||K^n[g] - K_{approx}^n[g]||

# Fix the number of observables used to make 
N_reconst = 30

common_obs = np.array(list(set(mixed_idx[range(N_reconst)]) & set(ranked_idx[range(N_reconst)]))) # list of indices of common observables
print(common_obs)

K_sub_mixed = K_approx_one[np.ix_(mixed_idx[range(N_reconst)],mixed_idx[range(N_reconst)])]
K_sub_PR = K_approx_one[np.ix_(ranked_idx[range(N_reconst)],ranked_idx[range(N_reconst)])]

M_error = 1000
# dataX_for_angle = np.random.default_rng().exponential(scale=1, size=[d,M_ang])
# dataY_for_angle = np.random.default_rng().exponential(scale=1, size=[d,M_ang])
dataX_for_error = np.random.default_rng(14).uniform(-2, 2, size=(d, M_error))

Psi_X_max_budget = np.zeros([M_error,N])
Psi_X_mixed = np.zeros([M_error,N_reconst])
Psi_X_PR = np.zeros([M_error,N_reconst])

for i in range(M_error):
    for j in range(N):
        Psi_X_max_budget[i,j] = kp.psi_radial(dataX_for_error[:,i], Cj[j,:], 1/gamma_inv_scale)
    for j in range(N_reconst):
        l1 = mixed_idx[j]
        l2 = ranked_idx[j]
        Psi_X_mixed[i,j] = kp.psi_radial(dataX_for_error[:,i], Cj[l1,:], 1/gamma_inv_scale)
        Psi_X_PR[i,j] = kp.psi_radial(dataX_for_error[:,i], Cj[l2,:], 1/gamma_inv_scale)

# Visualize the subsets of observables to be compared.

# plt.scatter(y_mat_total[0,:],y_mat_total[1,:])
plt.scatter(x_mat_total[0,:],x_mat_total[1,:], marker = 'o', color = 'tab:gray')
plt.scatter(Cj[np.ix_(ranked_idx[range(N_reconst)], range(1))],Cj[np.ix_(ranked_idx[range(N_reconst)], range(1,2))], color = "tab:orange")
plt.scatter(Cj[np.ix_(mixed_idx[range(N_reconst)], range(1))],Cj[np.ix_(mixed_idx[range(N_reconst)], range(1,2))], color = "deepskyblue", marker = 'o', facecolors='none')
plt.scatter(Cj[np.ix_(common_obs, range(1))],Cj[np.ix_(common_obs, range(1,2))], color = "tab:red", marker = 'x')
for tt in common_obs:
    plt.text(Cj[tt,0]-0.1,Cj[tt, 1]+0.2, str(tt), fontsize=12, color = "tab:red")

In [ ]:
# Pick one `g_test' of the common observables

g_test_id = 80
# for g_test_id in common_obs:
#     break

# We need to find the index label corresponding to the chosen observable g_test from mixed_idx and ranked_idx, respectively.
mix_idx = np.where(mixed_idx == g_test_id)[0][0]
PR_idx = np.where(ranked_idx == g_test_id)[0][0]

dataY_for_error = dataX_for_error
Psi_Y_max_budget = Psi_X_max_budget
Psi_Y_mixed = Psi_X_mixed
Psi_Y_PR = Psi_X_PR
g_eval_full = np.zeros(M_error)

evolve_max = 10
error_list_max_budget = np.zeros(evolve_max)
error_list_mixed = np.zeros(evolve_max)
error_list_PR = np.zeros(evolve_max)

for n_evolve in range(evolve_max):
    for i in range(M_error):
        dataY_for_error[:,i] = kp.Duffing(dataY_for_error[:,i], alpha0=alpha0, beta0 = beta0, delta0 = delta0, dt = dt)
        g_eval_full[i] = kp.psi_radial(dataY_for_error[:,i], Cj[g_test_id,:], 1/gamma_inv_scale)
    Psi_Y_max_budget = np.matmul(Psi_Y_max_budget, K_approx_one)
    Psi_Y_mixed = np.matmul(Psi_Y_mixed, K_sub_mixed)
    Psi_Y_PR = np.matmul(Psi_Y_PR, K_sub_PR)

    g_eval_max_budget = Psi_Y_max_budget[:,g_test_id]
    g_eval_mixed = Psi_Y_mixed[:,mix_idx]
    g_eval_PR = Psi_Y_PR[:,PR_idx]

    error_list_max_budget[n_evolve] = np.linalg.norm(g_eval_max_budget - g_eval_full)
    error_list_mixed[n_evolve] = np.linalg.norm(g_eval_mixed - g_eval_full)
    error_list_PR[n_evolve] = np.linalg.norm(g_eval_PR - g_eval_full)

plt.figure()
plt.semilogy(error_list_max_budget, color = "tab:gray")
plt.xlabel("Evolution steps n")
plt.ylabel("K^n[g]-K_approx^n[g]")
plt.semilogy(error_list_mixed, color = "tab:blue")
plt.semilogy(error_list_PR, color = "tab:orange")
# plt.xlabel("Basis functions")
# plt.ylabel("PageRank")
plt.legend(["Max budget", "mixed", "PageRank"])
plt.xticks(range(evolve_max))
plt.title('Evolution errors for Obs. '+ str(g_test_id))
plt.grid(True)
plt.show()
